In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
from LDOI import qm
import pandas as pd
import numpy as np
import networkx as nx
import concurrent.futures
import par_helper as ph
from collections import deque

In [2]:
# def par_get_func(Gread, prefix, suffix, equal_sign, node):
#     ON_list=[x for x in Gread.nodes[node]['update_rules'].keys() if Gread.nodes[node]['update_rules'][x]==1]
#     rule=BDOIp.Getfunc(Gread.nodes[node]['update_nodes'],node,ON_list,prefix=prefix,suffix=suffix,equal_sign=equal_sign)
#     OFF_list=[x for x in Gread.nodes[node]['update_rules'].keys() if Gread.nodes[node]['update_rules'][x]==0]
#     negation_rule='~'+BDOIp.Getfunc(Gread.nodes[node]['update_nodes'],node,OFF_list,prefix=prefix,suffix=suffix,equal_sign=equal_sign)

#     return { 'rule': rule,
#              'negation_rule': negation_rule}

# def par_get_expanded_network(Gread,prefix='n',suffix='n',equal_sign='*='):
#     '''
#     paralleized by jijoo @ 210121
#     Return the expanded network for a given Boolean network model.
#     The Boolean network model is a DiGraph object in the output format of form_network().
#     The Boolean network model can be generated through form_network function by reading a text file in the Booleannet format.
#     The Boolean rules will first be converted to a disjuctive normal form before generating the expanded network.

#     Parameters
#     ----------
#     Gread     : the given Boolean network model
#     prefix='n': prefix to encode the node name to avoid one node's name is a part of another node's name
#     suffix='n': suffix to encode the node name to avoid one node's name is a part of another node's name
#               e.g. node name '1' will become 'n1n' in the returned result
#     equal_sign: the equal sign of the rule in the returned result, whose default value follows the Booleannet format

#     Returns
#     -------
#     The expanded network for the given Boolean network model.


#     '''
#     G_expand=nx.DiGraph()
#     rules = []
#     #first write rules for negation nodes
#     negation_rules=[]
#     expanded_nodes=set()
        
    
#     with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
#         futures = {executor.submit(par_get_func, Gread.copy(), prefix, suffix, equal_sign, n): n for n in Gread.nodes()}
    
#     for future in concurrent.futures.as_completed(futures):
#         node = futures[future]
# #         print(str(node)+' DONE')
#         temp = future.result()
#         temp_rule = temp['rule']
#         temp_negation_rule = temp['negation_rule']
#         rules.append(temp_rule)
#         negation_rules.append(temp_negation_rule)
#         expanded_nodes.add(temp_rule.split('*=')[0])
#         expanded_nodes.add(temp_negation_rule.split('*=')[0])
        
# #     for node in Gread.nodes():
# #         ON_list=[x for x in Gread.nodes[node]['update_rules'].keys() if Gread.nodes[node]['update_rules'][x]==1]
# #         rule=Getfunc(Gread.nodes[node]['update_nodes'],node,ON_list,prefix=prefix,suffix=suffix,equal_sign=equal_sign)
# #         OFF_list=[x for x in Gread.nodes[node]['update_rules'].keys() if Gread.nodes[node]['update_rules'][x]==0]
# #         negation_rule='~'+Getfunc(Gread.nodes[node]['update_nodes'],node,OFF_list,prefix=prefix,suffix=suffix,equal_sign=equal_sign)
# #         rules.append(rule)
# #         negation_rules.append(negation_rule)
# #         expanded_nodes.add(rule.split('*=')[0])
# #         expanded_nodes.add(negation_rule.split('*=')[0])
#     #then for each line in the rules, construct Boolean network
#     composite_nodes=[]
#     rules.extend(negation_rules)
# #     print('parallel DONE')
#     for line in rules:
#         child, update_rule=line.split('*=')
#         update_rule=update_rule.strip()
#         if update_rule[0]=='(' and update_rule[-1]==')':
#             update_rule=update_rule[1:-1]
#         #single parent situation
#         if child[0]=='~':
#             normal_child=child[1:]
#         else:
#             normal_child=child[:]
#         normal_child=normal_child[len(prefix):len(normal_child)-len(suffix)]
#         #deal with source node situation
#         if not Gread.nodes[int(normal_child)]['update_nodes']:
#             G_expand.add_node(child)     #maybe this do not need to be done
#         else:
#             if 'or' in update_rule:
#                 parents=update_rule.split(' or ')
#             else:
#                 parents=[update_rule]
#             parents.sort()
#             for parent in parents:
#                 parent=parent.replace('not ','~').replace('(','').replace(')','')
#                 if 'and' in parent:
#                     composite_node=parent.replace(' and ','_')
#                     composite_nodes.append(composite_node)
#                     G_expand.add_edge(composite_node,child)
#                     for component in composite_node.split('_'):
#                         G_expand.add_edge(component,composite_node)
#                 else:
#                     G_expand.add_edge(parent,child)
#     return G_expand.copy()

In [55]:
Model = jR.fibroblast()
BooleanRuleFileName = '../'+Model['BooleanRule_filename']
NetworkName = Model['network_name']

NumInputs = Model['num_inputs']
NumInputConditions = Model['num_input_conditions']

InputConditions = Model['input_conditions']

OutputNodes = Model['output_nodes']
InputNodes = Model['input_nodes']

# Set parameters
# Note the node name for Gread is the index (integer), one can encode the nodename by adding prefix and suffix
# If the node name from the input file is not this simple, one need to create a dictionary to record the mapping
Prefix, Suffix = 'n', 'n'

TempI = jR.initialize(BooleanRuleFileName, Prefix, Suffix)
Mapping = TempI['mapping']
InverseMapping = TempI['inverse_mapping']
GRead = TempI['Gread']
ReadNodes = TempI['read_nodes']
if Model['reduction_required']:
    GRead = BDOIp.Get_reduced_network(GRead)

# form expanded network
# GExpanded = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
# nx.write_gml(GExpanded, NetworkName + '_expanded_network.gml')
# try:
#     GExpanded = nx.read_gml('../networks/' + NetworkName + '_expanded_network.gml')
# except IOError:
# GExpanded = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
# GExpanded = par_get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
#     nx.write_gml(GExpanded, '../networks/' + NetworkName + '_expanded_network.gml')

In [56]:
%timeit BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)

2h 31min 44s ± 48min 51s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%timeit ph.par_get_expanded_network(GRead, prefix=Prefix, suffix=Suffix, worker=5)

1h 40min 34s ± 20 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%timeit ph.par_get_expanded_network2(GRead, prefix=Prefix, suffix=Suffix, worker=5)

1h 40min 2s ± 8.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%timeit ph.par_get_expanded_network3(GRead, prefix=Prefix, suffix=Suffix, worker=5)

1h 40min 19s ± 1min 8s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%timeit ph.par_get_expanded_network2(GRead, prefix=Prefix, suffix=Suffix, worker=10)

1h 43min 33s ± 24.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
%timeit ph.par_get_expanded_network2(GRead, prefix=Prefix, suffix=Suffix, worker=2)

1h 42min 3s ± 1min 50s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
single = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
par = ph.par_get_expanded_network2(GRead, prefix=Prefix, suffix=Suffix, worker=5)
print(nx.is_isomorphic(single, par))